In [36]:
import os
import numpy as np

def write_gcode(gcode_content, gcode_path, heatbed_temp=37):
    """
    Convert list of gcommands into .gcode file. Also add start and end commands

    gcommand_layers, list of lists of GCommand objects
    gcode_path, path to write output
    heatbed_temp, start temp UNUSED
    """
    assert heatbed_temp <= 200, "{} > max temp 200".format(heatbed_temp)

    with open(gcode_path, 'w') as gcode_file:
        for i in range(len(gcode_content)):
            gcode_file.write(gcode_content[i])

def TCommand(content, lines, psi):
    """
    Returns a gcode representation of the command to initialize each extruder element
    """
    if   "T0" in content:
        return str("M42 P4 S" + str(psi) + "\n")
    elif "T1" in content:
        return str("M42 P5 S" + str(psi) + "\n")
    elif "T2" in content:
        return str("M42 P6 S" + str(psi) + "\n")
#     elif "T3" in content:
#         return str("M42 P6 S" + str(psi) + "\n")
#     elif "T4" in content:
#         return str("M42 P6 S" + str(psi) + "\n")
    else:
        return ""

def main():
    psi_on  = 15
    psi_off = 255
    counter = 0
    
    lines = []
    stack = []
    
    with open('STL/20mm_cubeSlic3r.gcode') as f:
        content = f.readlines()
        
    for i in range(len(content)):
        #Count Each Layer
        if content[i] != "\r\n":
            if ";After" in content[i]:
                lines.append(";LAYER:" + str(counter) + "\n")
                counter += 1
            lines.append(content[i])

        if (i + 1 < len(content)):
            if ("T" in content[i+1]) or (content[i+1] in ["\n", "\r\n"]):
                if stack:
                    lines.append(stack.pop())
                stack.append(TCommand(content[i+1], lines, psi_off))
            else:
                lines.append(TCommand(content[i], lines, psi_on))
    write_gcode(lines, '20mm_cube_out.gcode')

if __name__ == '__main__':
    main()

